In [14]:
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np

In [6]:
df = pd.read_parquet("datasets/df_portfolio_train.parquet")
train_aapl = df[df["tic"] == "AAPL"]

In [7]:
train_aapl

,date,tic,close,high,low,High_VIX,Low_VIX,Close_VIX,High_SPY,Low_SPY,...,ics,TTM Revenue(Billion),TTM EBITDA(Billion),EBITDA Margin,Price to Book Ratio,PE Ratio,Price to Sales Ratio,Assets,NetIncomeLoss,Return on Equity(%)
0,2011-01-03,AAPL,0.050363,0.059087,0.058891,0.210015,0.240301,0.212964,0.265844,0.263931,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
10,2011-01-04,AAPL,0.050626,0.059488,0.059491,0.213408,0.246270,0.210183,0.265365,0.264960,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
20,2011-01-05,AAPL,0.051040,0.059817,0.059735,0.210015,0.239591,0.205829,0.266094,0.265527,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
30,2011-01-06,AAPL,0.050999,0.059980,0.060352,0.205452,0.238596,0.210424,0.266324,0.266682,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
40,2011-01-07,AAPL,0.051364,0.060177,0.060170,0.211419,0.235470,0.207280,0.266199,0.264876,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25110,2020-12-23,AAPL,0.649575,0.663410,0.663858,0.277056,0.314481,0.281896,0.770074,0.771049,...,0.795858,0.696168,0.592054,0.782935,0.587847,0.986924,0.905237,0.0,0.0,-0.346913
25120,2020-12-24,AAPL,0.654585,0.668570,0.665482,0.267111,0.303965,0.260370,0.768845,0.771532,...,0.795858,0.696168,0.592054,0.782935,0.587847,0.986924,0.905237,0.0,0.0,-0.346913
25130,2020-12-28,AAPL,0.677996,0.688007,0.677716,0.258804,0.300554,0.262426,0.776261,0.779133,...,0.795858,0.696168,0.592054,0.782935,0.587847,0.986924,0.905237,0.0,0.0,-0.346913
25140,2020-12-29,AAPL,0.668969,0.695271,0.681929,0.277524,0.298281,0.279115,0.779199,0.778629,...,0.795858,0.696168,0.592054,0.782935,0.587847,0.986924,0.905237,0.0,0.0,-0.346913


In [16]:
y = train_aapl['close']
X = train_aapl.drop(columns=["date","tic","high","low","close"])

In [19]:
from sklearn.linear_model import LassoCV

lasso_cv = LassoCV(cv=5).fit(X, y)
best_alpha = lasso_cv.alpha_
print("Best alpha value from cross-validation:", best_alpha)

# Refit Lasso with the best alpha
lasso_best = Lasso(alpha=best_alpha)
lasso_best.fit(X, y)

# Get the coefficients of the features
coefficients = lasso_best.coef_

# Identify the selected features
selected_features = X.columns[coefficients != 0]
print("Selected Features:", selected_features)

Best alpha value from cross-validation: 0.00037985750571490484
Selected Features: Index(['High_SPY', 'Close_SPY', 'Fear Greed', 'ratingScore',
       'pound_dollar_exchange_rate', 'unemployment', 'EBITDA Margin',
       'Price to Book Ratio', 'PE Ratio', 'Return on Equity(%)'],
      dtype='object')
